In [1]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("cpu")
    print("Using MPS")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using GPU")
else:
    device = torch.device("cpu")
    print("No GPU/MPS available, falling back to CPU.")

Using GPU


In [2]:
""" from mesh_video_generator import MeshVideoGenerator

hw = 32
num_views = 2
data_dirs = ["meshes", "SHREC19_MH_dataset", "SHREC20b_hires/models", "MPI-FAUST/training/scans"]

generator = MeshVideoGenerator(
    output_dir=data_dirs[0],
    hw=hw,
    num_views=num_views,
    use_normal_map=True,
    device=device
) """
#generator.process_folder("meshes", display_frames=False)
#generator.process_single_mesh("meshes", index=5)

' from mesh_video_generator import MeshVideoGenerator\n\nhw = 32\nnum_views = 2\ndata_dirs = ["meshes", "SHREC19_MH_dataset", "SHREC20b_hires/models", "MPI-FAUST/training/scans"]\n\ngenerator = MeshVideoGenerator(\n    output_dir=data_dirs[0],\n    hw=hw,\n    num_views=num_views,\n    use_normal_map=True,\n    device=device\n) '

In [3]:
#import torch
from diff3f import get_features_per_vertex
from diffusion import init_pipe

from utils import convert_mesh_container_to_torch_mesh, cosine_similarity, double_plot, get_colors
from dataloaders.mesh_container import MeshContainer
from dino import init_dino
from sam2_setup import init_sam2
#from functional_map import compute_surface_map

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/workspace/miniconda3/envs/diff3f/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [4]:
num_views = 2
H = 32
W = 32
num_images_per_prompt = 1
tolerance = 0.004
random_seed = 42
use_normal_map = True
is_tosca = False

bq = True
use_sam = True
use_only_diffusion = False
use_diffusion = False

In [5]:
def compute_features(device, sam_model, dino_model, pipe, m, prompt, num_views, H, W, tolerance):
    mesh = convert_mesh_container_to_torch_mesh(m, device=device, is_tosca=False)
    mesh_vertices = mesh.verts_list()[0]

    features = get_features_per_vertex(
        device=device,
        sam_model=sam_model,
        pipe=pipe,
        dino_model=dino_model,
        mesh=mesh,
        prompt=prompt,
        num_views=num_views,
        H=H,
        W=W,
        tolerance=tolerance,
        use_normal_map= use_normal_map,
        num_images_per_prompt=num_images_per_prompt,
        mesh_vertices=mesh_vertices,
        bq=bq,
        use_sam = use_sam,
        use_only_diffusion = use_only_diffusion,
        use_diffusion = use_diffusion,
    )
    return features.cpu()

In [6]:
sam_model = init_sam2(device)
pipe = init_pipe(device)
dino_model = init_dino(device)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb14/dinov2_vitb14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitb14_pretrain.pth
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330M/330M [00:02<00:0

In [7]:
source_file_path = "meshes/cow.obj"
target_file_path = "meshes/camel.obj"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)

In [8]:
f_source = compute_features(device, sam_model, dino_model, pipe, source_mesh, "cow", num_views, H, W, tolerance)
f_target = compute_features(device, sam_model, dino_model, pipe, target_mesh, "camel", num_views, H, W, tolerance)

Starting batch_render with num_views=2, H=32, W=32
Rendering completed successfully
Starting batch_render with num_views=2, H=32, W=32
Rendering completed successfully
batched_renderings shape:  torch.Size([4, 32, 32, 4])
normal_batched_renderings shape:  torch.Size([4, 32, 32, 1, 3])
camera R: tensor([[[-1.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -1.0000e+00]],

        [[ 1.0000e+00, -0.0000e+00,  8.7423e-08],
         [ 0.0000e+00,  1.0000e+00,  0.0000e+00],
         [-8.7423e-08,  0.0000e+00,  1.0000e+00]],

        [[-1.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  1.0000e+00, -1.7453e-05],
         [-0.0000e+00, -1.7453e-05, -1.0000e+00]],

        [[-1.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  1.0000e+00,  1.7468e-05],
         [ 0.0000e+00,  1.7468e-05, -1.0000e+00]]], device='cuda:0')
camera T: tensor([[-0.0000, -0.0000, 1.0169],
        [-0.0000, -0.0000, 1.0169

/workspace/miniconda3/envs/diff3f/lib/python3.10/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1729805341246/work/aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Video saved to output.mp4



  0%|                                                                                                                                                                   | 0/4 [00:06<?, ?it/s]


RuntimeError: The size of tensor a (32) must match the size of tensor b (2) at non-singleton dimension 3

In [ ]:
s = cosine_similarity(f_source.to(device),f_target.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[s]
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)